# Lab 2.6.6: Image Generation Pipeline - End-to-End System

**Module:** 2.6 - Diffusion Models  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Build a complete image generation pipeline
- [ ] Implement seed management for reproducibility
- [ ] Add batch generation with variations
- [ ] Create a prompt templating system
- [ ] Add automatic captioning of outputs
- [ ] (Optional) Build a Gradio interface

---

## 📚 Prerequisites

- Completed: Labs 2.6.1-2.6.5
- Knowledge of: All previous diffusion model concepts
- Installed: `diffusers`, `transformers`, `gradio` (optional)

---

## 🌍 Real-World Context

**Production image generation systems need:**

- **Reproducibility**: Same seed = same image
- **Batch processing**: Generate many images efficiently
- **Variations**: Explore different versions of a concept
- **Metadata**: Track what was generated and how
- **UI**: Let non-technical users interact with the system

This lab combines everything you've learned into a reusable pipeline!

---

## Part 1: Setting Up the Pipeline

In [ ]:
# Core imports
import torch
import gc
import time
import json
import random
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, asdict
from typing import List, Optional, Dict, Any

# Diffusers
from diffusers import (
    StableDiffusionXLPipeline,
    DPMSolverMultistepScheduler,
)

# Image handling
from PIL import Image
import matplotlib.pyplot as plt

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

In [ ]:
@dataclass
class GenerationConfig:
    """
    Configuration for image generation.
    
    All parameters needed to reproduce a generation.
    """
    prompt: str
    negative_prompt: str = "blurry, low quality, distorted, ugly, deformed"
    width: int = 1024
    height: int = 1024
    num_inference_steps: int = 30
    guidance_scale: float = 7.5
    seed: Optional[int] = None
    
    def __post_init__(self):
        if self.seed is None:
            self.seed = random.randint(0, 2**32 - 1)
    
    def to_dict(self) -> Dict:
        return asdict(self)
    
    @classmethod
    def from_dict(cls, data: Dict) -> 'GenerationConfig':
        return cls(**data)


@dataclass
class GenerationResult:
    """
    Result of an image generation.
    
    Contains the image and metadata for reproducibility.
    """
    image: Image.Image
    config: GenerationConfig
    generation_time: float
    timestamp: str = None
    
    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now().isoformat()
    
    def save(self, output_dir: str, name: str = None):
        """Save image and metadata."""
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
        
        if name is None:
            name = f"gen_{self.config.seed}"
        
        # Save image
        image_path = output_dir / f"{name}.png"
        self.image.save(image_path)
        
        # Save metadata
        metadata = {
            'config': self.config.to_dict(),
            'generation_time': self.generation_time,
            'timestamp': self.timestamp,
        }
        metadata_path = output_dir / f"{name}.json"
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        return image_path, metadata_path


print("Data classes defined!")

In [ ]:
class ImageGenerationPipeline:
    """
    A complete image generation pipeline.
    
    Features:
    - Reproducible generation with seeds
    - Batch generation
    - Variation generation
    - Automatic metadata saving
    - Prompt templates
    
    Example:
        pipeline = ImageGenerationPipeline()
        
        # Single image
        result = pipeline.generate("a beautiful sunset")
        
        # Batch with variations
        results = pipeline.generate_batch(
            "a cat",
            num_images=4,
            variation_mode="seed",
        )
    """
    
    def __init__(
        self,
        model_id: str = "stabilityai/stable-diffusion-xl-base-1.0",
        dtype: torch.dtype = torch.bfloat16,
    ):
        self.model_id = model_id
        self.dtype = dtype
        self.pipe = None
        self.is_loaded = False
        
        # Prompt templates
        self.templates = {
            'photorealistic': "{subject}, professional photography, 8K resolution, sharp focus, natural lighting",
            'artistic': "{subject}, digital art, trending on artstation, highly detailed, vibrant colors",
            'anime': "{subject}, anime style, Studio Ghibli inspired, beautiful, detailed",
            'oil_painting': "{subject}, oil painting, classical style, rich colors, masterpiece",
            'watercolor': "{subject}, watercolor painting, soft colors, artistic, delicate brushstrokes",
            'fantasy': "{subject}, fantasy art, magical, ethereal lighting, epic, detailed",
            'cyberpunk': "{subject}, cyberpunk style, neon lights, futuristic, dark, atmospheric",
            'minimalist': "{subject}, minimalist style, simple, clean, modern, elegant",
        }
        
        # Default negative prompts by style
        self.negative_templates = {
            'photorealistic': "cartoon, anime, illustration, painting, drawing, art, 3D render",
            'artistic': "photo, photograph, realistic, 3D, render",
            'default': "blurry, low quality, distorted, ugly, deformed, watermark, text",
        }
    
    def load_model(self):
        """Load the diffusion model."""
        if self.is_loaded:
            return
        
        print(f"Loading {self.model_id}...")
        start = time.time()
        
        self.pipe = StableDiffusionXLPipeline.from_pretrained(
            self.model_id,
            torch_dtype=self.dtype,
            variant="fp16",
            use_safetensors=True,
        )
        self.pipe = self.pipe.to(device)
        self.pipe.scheduler = DPMSolverMultistepScheduler.from_config(
            self.pipe.scheduler.config
        )
        
        self.is_loaded = True
        print(f"✅ Loaded in {time.time() - start:.1f}s")
    
    def unload_model(self):
        """Unload model to free memory."""
        if self.pipe:
            del self.pipe
            self.pipe = None
        self.is_loaded = False
        gc.collect()
        torch.cuda.empty_cache()
        print("Model unloaded")
    
    def apply_template(self, subject: str, style: str = None) -> str:
        """Apply a prompt template to a subject."""
        if style is None or style not in self.templates:
            return subject
        return self.templates[style].format(subject=subject)
    
    def get_negative_prompt(self, style: str = None) -> str:
        """Get appropriate negative prompt for style."""
        if style in self.negative_templates:
            return self.negative_templates[style] + ", " + self.negative_templates['default']
        return self.negative_templates['default']
    
    def generate(
        self,
        prompt: str,
        style: str = None,
        config: GenerationConfig = None,
        **kwargs,
    ) -> GenerationResult:
        """
        Generate a single image.
        
        Args:
            prompt: Text prompt or subject (if using template)
            style: Optional style template to apply
            config: Optional full configuration
            **kwargs: Override config parameters
        
        Returns:
            GenerationResult with image and metadata
        """
        self.load_model()
        
        # Apply template if style specified
        full_prompt = self.apply_template(prompt, style)
        negative = self.get_negative_prompt(style)
        
        # Create or update config
        if config is None:
            config = GenerationConfig(
                prompt=full_prompt,
                negative_prompt=negative,
                **kwargs,
            )
        
        # Generate
        generator = torch.Generator(device=device).manual_seed(config.seed)
        
        torch.cuda.synchronize()
        start = time.time()
        
        image = self.pipe(
            prompt=config.prompt,
            negative_prompt=config.negative_prompt,
            width=config.width,
            height=config.height,
            num_inference_steps=config.num_inference_steps,
            guidance_scale=config.guidance_scale,
            generator=generator,
        ).images[0]
        
        torch.cuda.synchronize()
        gen_time = time.time() - start
        
        return GenerationResult(
            image=image,
            config=config,
            generation_time=gen_time,
        )
    
    def generate_batch(
        self,
        prompt: str,
        num_images: int = 4,
        style: str = None,
        variation_mode: str = "seed",
        base_seed: int = None,
        **kwargs,
    ) -> List[GenerationResult]:
        """
        Generate multiple images.
        
        Args:
            prompt: Text prompt or subject
            num_images: Number of images to generate
            style: Optional style template
            variation_mode: "seed" (different seeds) or "guidance" (different guidance)
            base_seed: Starting seed for variations
            **kwargs: Additional config parameters
        
        Returns:
            List of GenerationResults
        """
        if base_seed is None:
            base_seed = random.randint(0, 2**32 - 1)
        
        results = []
        
        for i in range(num_images):
            print(f"Generating {i+1}/{num_images}...")
            
            if variation_mode == "seed":
                seed = base_seed + i
                guidance = kwargs.get('guidance_scale', 7.5)
            elif variation_mode == "guidance":
                seed = base_seed
                guidances = [5.0, 7.0, 9.0, 12.0]
                guidance = guidances[i % len(guidances)]
            else:
                seed = base_seed + i
                guidance = kwargs.get('guidance_scale', 7.5)
            
            result = self.generate(
                prompt=prompt,
                style=style,
                seed=seed,
                guidance_scale=guidance,
                **kwargs,
            )
            results.append(result)
        
        return results
    
    def generate_style_comparison(
        self,
        subject: str,
        styles: List[str] = None,
        seed: int = None,
    ) -> Dict[str, GenerationResult]:
        """
        Generate the same subject in different styles.
        
        Args:
            subject: Subject description (without style)
            styles: List of styles to try (uses all if None)
            seed: Seed for consistency
        
        Returns:
            Dict mapping style name to GenerationResult
        """
        if styles is None:
            styles = list(self.templates.keys())
        
        if seed is None:
            seed = random.randint(0, 2**32 - 1)
        
        results = {}
        for style in styles:
            print(f"Generating {style} style...")
            result = self.generate(subject, style=style, seed=seed)
            results[style] = result
        
        return results


print("ImageGenerationPipeline defined!")

---

## Part 2: Using the Pipeline

In [ ]:
# Initialize the pipeline
pipeline = ImageGenerationPipeline()

print("\nAvailable style templates:")
for name, template in pipeline.templates.items():
    print(f"  {name}: {template[:50]}...")

In [ ]:
# Single generation with template
result = pipeline.generate(
    "a majestic lion",
    style="photorealistic",
    seed=42,
)

print(f"\nGeneration complete!")
print(f"  Prompt: {result.config.prompt}")
print(f"  Seed: {result.config.seed}")
print(f"  Time: {result.generation_time:.2f}s")

plt.figure(figsize=(10, 10))
plt.imshow(result.image)
plt.title(f"Seed: {result.config.seed}")
plt.axis('off')
plt.show()

In [ ]:
# Batch generation with seed variations
results = pipeline.generate_batch(
    "a cozy cabin in the woods",
    num_images=4,
    style="artistic",
    variation_mode="seed",
    base_seed=100,
)

# Display all results
fig, axes = plt.subplots(2, 2, figsize=(14, 14))
for ax, result in zip(axes.flatten(), results):
    ax.imshow(result.image)
    ax.set_title(f"Seed: {result.config.seed}")
    ax.axis('off')
plt.suptitle("Seed Variations", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Style comparison
style_results = pipeline.generate_style_comparison(
    "a serene Japanese garden",
    styles=["photorealistic", "watercolor", "anime", "cyberpunk"],
    seed=42,
)

# Display
fig, axes = plt.subplots(2, 2, figsize=(14, 14))
for ax, (style, result) in zip(axes.flatten(), style_results.items()):
    ax.imshow(result.image)
    ax.set_title(f"{style} ({result.generation_time:.1f}s)")
    ax.axis('off')
plt.suptitle("Style Comparison: Same Subject, Different Styles", fontsize=14)
plt.tight_layout()
plt.show()

---

## Part 3: Saving and Reproducing Results

In [ ]:
# Save results with metadata
output_dir = Path("./pipeline_output")
output_dir.mkdir(exist_ok=True)

# Generate and save
result = pipeline.generate(
    "a mystical dragon",
    style="fantasy",
    seed=12345,
)

image_path, metadata_path = result.save(output_dir, "dragon_fantasy")

print(f"Saved to:")
print(f"  Image: {image_path}")
print(f"  Metadata: {metadata_path}")

# Show saved metadata
print(f"\nMetadata contents:")
with open(metadata_path) as f:
    print(json.dumps(json.load(f), indent=2))

In [ ]:
# Reproduce from metadata
def reproduce_from_metadata(metadata_path: str, pipeline: ImageGenerationPipeline) -> GenerationResult:
    """Reproduce an image from saved metadata."""
    with open(metadata_path) as f:
        metadata = json.load(f)
    
    config = GenerationConfig.from_dict(metadata['config'])
    return pipeline.generate(config=config, prompt=config.prompt)

# Reproduce the dragon
reproduced = reproduce_from_metadata(metadata_path, pipeline)

# Compare original and reproduced
fig, axes = plt.subplots(1, 2, figsize=(14, 7))
axes[0].imshow(result.image)
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(reproduced.image)
axes[1].set_title("Reproduced from Metadata")
axes[1].axis('off')

plt.suptitle("Reproducibility Test", fontsize=14)
plt.tight_layout()
plt.show()

print("\n✅ Same seed = identical image!")

---

## Part 4: Custom Prompt Templates

In [ ]:
class PromptBuilder:
    """
    Advanced prompt builder with modular components.
    
    Builds prompts from:
    - Subject
    - Style
    - Lighting
    - Camera angle
    - Quality modifiers
    """
    
    STYLES = {
        'photo': 'professional photography',
        'oil': 'oil painting',
        'watercolor': 'watercolor painting',
        'digital': 'digital art',
        'anime': 'anime style',
        '3d': '3D render',
        'sketch': 'pencil sketch',
    }
    
    LIGHTING = {
        'golden': 'golden hour lighting',
        'studio': 'studio lighting',
        'dramatic': 'dramatic lighting',
        'soft': 'soft natural lighting',
        'neon': 'neon lights',
        'moonlight': 'moonlit scene',
    }
    
    CAMERA = {
        'closeup': 'close-up shot',
        'wide': 'wide angle shot',
        'portrait': 'portrait shot',
        'aerial': 'aerial view',
        'macro': 'macro photography',
    }
    
    QUALITY = {
        'basic': 'detailed',
        'good': 'highly detailed, sharp focus',
        'best': 'highly detailed, 8K resolution, sharp focus, professional',
    }
    
    @classmethod
    def build(
        cls,
        subject: str,
        style: str = 'photo',
        lighting: str = None,
        camera: str = None,
        quality: str = 'good',
        extras: List[str] = None,
    ) -> str:
        """
        Build a complete prompt from components.
        
        Example:
            prompt = PromptBuilder.build(
                subject="a red fox in a forest",
                style="photo",
                lighting="golden",
                camera="closeup",
                quality="best",
            )
        """
        parts = [subject]
        
        if style in cls.STYLES:
            parts.append(cls.STYLES[style])
        
        if lighting in cls.LIGHTING:
            parts.append(cls.LIGHTING[lighting])
        
        if camera in cls.CAMERA:
            parts.append(cls.CAMERA[camera])
        
        if quality in cls.QUALITY:
            parts.append(cls.QUALITY[quality])
        
        if extras:
            parts.extend(extras)
        
        return ", ".join(parts)


# Test the prompt builder
prompt = PromptBuilder.build(
    subject="a red fox in a forest",
    style="photo",
    lighting="golden",
    camera="closeup",
    quality="best",
)

print("Built prompt:")
print(f"  {prompt}")

In [ ]:
# Generate using the prompt builder
built_prompt = PromptBuilder.build(
    subject="a wise old owl perched on a branch",
    style="digital",
    lighting="moonlight",
    camera="portrait",
    quality="best",
    extras=["mystical atmosphere", "stars in background"],
)

print(f"Prompt: {built_prompt}\n")

result = pipeline.generate(
    built_prompt,
    seed=42,
)

plt.figure(figsize=(10, 10))
plt.imshow(result.image)
plt.title("Generated with PromptBuilder")
plt.axis('off')
plt.show()

---

## Part 5: Automatic Captioning (Optional)

Let's add automatic description generation for our outputs.

In [ ]:
# Try to load BLIP for captioning (optional)
try:
    from transformers import BlipProcessor, BlipForConditionalGeneration
    
    class ImageCaptioner:
        """Generate captions for images using BLIP."""
        
        def __init__(self):
            self.processor = BlipProcessor.from_pretrained(
                "Salesforce/blip-image-captioning-base"
            )
            self.model = BlipForConditionalGeneration.from_pretrained(
                "Salesforce/blip-image-captioning-base",
                torch_dtype=torch.bfloat16,
            ).to(device)
        
        def caption(self, image: Image.Image) -> str:
            """Generate a caption for an image."""
            inputs = self.processor(image, return_tensors="pt").to(device)
            
            with torch.no_grad():
                output = self.model.generate(**inputs, max_new_tokens=50)
            
            return self.processor.decode(output[0], skip_special_tokens=True)
    
    # Initialize captioner
    captioner = ImageCaptioner()
    print("✅ Image captioner loaded!")
    
    # Test captioning on our generated image
    caption = captioner.caption(result.image)
    print(f"\nGenerated caption: {caption}")
    
except ImportError:
    print("BLIP not available. Install with: pip install transformers")
    captioner = None

---

## Part 6: Gradio Interface (Optional)

Let's create a simple web interface for our pipeline.

In [ ]:
try:
    import gradio as gr
    
    def generate_interface(
        prompt: str,
        style: str,
        guidance: float,
        steps: int,
        seed: int,
    ):
        """Gradio interface function."""
        if seed == -1:
            seed = random.randint(0, 2**32 - 1)
        
        result = pipeline.generate(
            prompt=prompt,
            style=style if style != "none" else None,
            guidance_scale=guidance,
            num_inference_steps=steps,
            seed=seed,
        )
        
        info = f"Seed: {result.config.seed}\nTime: {result.generation_time:.1f}s"
        return result.image, info
    
    # Create Gradio interface
    demo = gr.Interface(
        fn=generate_interface,
        inputs=[
            gr.Textbox(label="Prompt", placeholder="Enter your prompt..."),
            gr.Dropdown(
                choices=["none"] + list(pipeline.templates.keys()),
                value="none",
                label="Style Template",
            ),
            gr.Slider(1, 15, value=7.5, label="Guidance Scale"),
            gr.Slider(10, 50, value=25, step=5, label="Steps"),
            gr.Number(value=-1, label="Seed (-1 for random)"),
        ],
        outputs=[
            gr.Image(label="Generated Image"),
            gr.Textbox(label="Generation Info"),
        ],
        title="Image Generation Pipeline",
        description="Generate images with SDXL on DGX Spark",
    )
    
    print("✅ Gradio interface created!")
    print("Run demo.launch() to start the web interface")
    
    # Uncomment to launch:
    # demo.launch(share=True)
    
except ImportError:
    print("Gradio not available. Install with: pip install gradio")

---

## Part 7: Complete Pipeline Demo

In [ ]:
# Full pipeline demonstration
print("=" * 60)
print("COMPLETE PIPELINE DEMONSTRATION")
print("=" * 60)

# 1. Build prompt with PromptBuilder
print("\n1. Building prompt...")
prompt = PromptBuilder.build(
    subject="a futuristic cityscape",
    style="digital",
    lighting="neon",
    quality="best",
    extras=["flying cars", "holographic billboards"],
)
print(f"   Prompt: {prompt[:70]}...")

# 2. Generate image
print("\n2. Generating image...")
result = pipeline.generate(prompt, seed=42)
print(f"   Time: {result.generation_time:.2f}s")

# 3. Save with metadata
print("\n3. Saving with metadata...")
image_path, metadata_path = result.save(output_dir, "futuristic_city")
print(f"   Saved to: {image_path}")

# 4. Caption (if available)
if captioner:
    print("\n4. Auto-captioning...")
    caption = captioner.caption(result.image)
    print(f"   Caption: {caption}")

# 5. Display
print("\n5. Displaying result...")
plt.figure(figsize=(12, 12))
plt.imshow(result.image)
plt.title(f"Seed: {result.config.seed} | Time: {result.generation_time:.1f}s")
plt.axis('off')
plt.show()

print("\n" + "=" * 60)
print("PIPELINE COMPLETE")
print("=" * 60)

---

## 🎉 Checkpoint - Module Complete!

You've now completed the entire Diffusion Models module:

### Labs Completed:
- ✅ **Lab 2.6.1**: Diffusion theory and DDPM from scratch
- ✅ **Lab 2.6.2**: SDXL generation and prompt engineering
- ✅ **Lab 2.6.3**: ControlNet for guided generation
- ✅ **Lab 2.6.4**: Flux architecture exploration
- ✅ **Lab 2.6.5**: LoRA training for custom styles
- ✅ **Lab 2.6.6**: Complete generation pipeline

### Skills Acquired:
- Understanding diffusion model theory
- Text-to-image generation with SDXL and Flux
- Structural control with ControlNet
- Custom style training with LoRA
- Building production-ready pipelines

---

## 🧹 Cleanup

In [ ]:
# Clean up all resources
pipeline.unload_model()

if captioner:
    del captioner

gc.collect()
torch.cuda.empty_cache()

print("All resources cleaned up!")
print("\n🎉 Congratulations on completing Module 2.6: Diffusion Models!")

---

## Next Steps

Now that you've mastered diffusion models, you're ready for:

**Domain 3: LLM Systems** - Module 3.1: LLM Fine-Tuning

You'll learn to fine-tune large language models using the same principles you learned here with LoRA!